In [1]:
# Reload automatically when the file is changed.
%load_ext autoreload
%autoreload 2

# Importing Modules

In [2]:
import os 
import sys
MODULES_PATH = "../../Modules/XG_Boost/"
MODELS_PATH = "../../Models/"

sys.path.append(MODULES_PATH)
sys.path.append(MODELS_PATH)


from Preprocessing import *
from Dataloaders_Preprocessing import *
from XG_Boost import *






# Importing Data

In [3]:
List_Data_Train = Load_Data('../../Data/atoms/train', '../../Data/energies/train.csv')
List_Data_Test= Load_Test_Data('../../Data/atoms/test')

In [4]:
Display_Molecule_From_Atom_List(List_Data_Train[0]['Atoms_List'], Width=800, Height=800, Background_Color='lightblue')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
# Dataset_Train = MoleculeDataset_Transformer(Data_List = List_Data_Train,
#                                             Nb_Atoms_Max_In_Molecule=23, Return_Energies=True, transform=None)

# Dataset_Val = MoleculeDataset_Transformer(Data_List = List_Data_Val,
#                                             Nb_Atoms_Max_In_Molecule=23, Return_Energies=False, transform=None)

Dataset_Train = MoleculeDataset_XGBoost(Data_List = List_Data_Train, Return_Energies=True)
from torch.utils.data import random_split

val_size = int(0.2 * len(Dataset_Train))
train_size = len(Dataset_Train) - val_size
Dataset_Train, Dataset_Val = random_split(Dataset_Train, [train_size, val_size], generator=torch.Generator().manual_seed(42))

Dataset_Test = MoleculeDataset_XGBoost(Data_List = List_Data_Test, Return_Energies=False)

IDs_Train,X_Train, y_Train =extract_data_for_xgboost(Dataset_Train)
IDs_Test,X_Test = extract_data_for_xgboost(Dataset_Test)
IDs_Val,X_Val, y_Val = extract_data_for_xgboost(Dataset_Val)


# Training the model

In [12]:
Model = XGBoostWrapper(
    input_dim=X_Train.shape[1], n_estimators=10000, max_depth=5, learning_rate=0.1, objective='reg:squarederror', model_type='regressor',reg_lambda=0)

2025-06-19 17:52:20,721 - INFO - XGBoostWrapper initialisé avec input_dim=141, model_type=regressor, reg_lambda=0, reg_alpha=0.0


In [13]:
Model.fit(X = X_Train, y = y_Train, early_stopping_rounds=1000,
        verbose=True,eval_set=[(X_Val, y_Val)])

2025-06-19 17:52:22,277 - INFO - Early stopping activé avec 1000 rounds


[0]	train-rmse:10.47582	eval-rmse:10.57124
[1]	train-rmse:9.43637	eval-rmse:9.52289
[2]	train-rmse:8.50162	eval-rmse:8.57906
[3]	train-rmse:7.66005	eval-rmse:7.73176
[4]	train-rmse:6.90356	eval-rmse:6.96730
[5]	train-rmse:6.22239	eval-rmse:6.28455
[6]	train-rmse:5.60866	eval-rmse:5.66068
[7]	train-rmse:5.05685	eval-rmse:5.10522
[8]	train-rmse:4.56043	eval-rmse:4.60961
[9]	train-rmse:4.11436	eval-rmse:4.15888
[10]	train-rmse:3.71311	eval-rmse:3.75122
[11]	train-rmse:3.35245	eval-rmse:3.39411
[12]	train-rmse:3.02831	eval-rmse:3.06286
[13]	train-rmse:2.73661	eval-rmse:2.77192
[14]	train-rmse:2.47259	eval-rmse:2.50776
[15]	train-rmse:2.23568	eval-rmse:2.26662
[16]	train-rmse:2.02290	eval-rmse:2.05557
[17]	train-rmse:1.83146	eval-rmse:1.86290
[18]	train-rmse:1.65948	eval-rmse:1.69284
[19]	train-rmse:1.50559	eval-rmse:1.53741
[20]	train-rmse:1.36787	eval-rmse:1.39958
[21]	train-rmse:1.24403	eval-rmse:1.27583
[22]	train-rmse:1.13257	eval-rmse:1.16570
[23]	train-rmse:1.03346	eval-rmse:1.06929


2025-06-19 17:52:28,488 - INFO - Entraînement terminé avec succès


In [8]:
Responses_Test,_ = Model.evaluate(X_Test)

In [9]:
Responses_Test

[-76.38921356201172,
 -67.019287109375,
 -72.83343505859375,
 -74.95111846923828,
 -89.90565490722656,
 -97.92659759521484,
 -78.07331848144531,
 -103.25520324707031,
 -83.47496795654297,
 -90.04190063476562,
 -74.5826416015625,
 -84.27063751220703,
 -84.1727523803711,
 -64.58111572265625,
 -76.25736999511719,
 -74.93040466308594,
 -83.08074188232422,
 -70.09066009521484,
 -91.06573486328125,
 -69.18962097167969,
 -83.03363037109375,
 -97.0693588256836,
 -74.66194915771484,
 -90.19943237304688,
 -76.04199981689453,
 -84.97735595703125,
 -71.84500122070312,
 -76.70137023925781,
 -72.78899383544922,
 -68.51243591308594,
 -83.69159698486328,
 -66.0852279663086,
 -83.38028717041016,
 -63.079833984375,
 -69.43285369873047,
 -56.75084686279297,
 -77.50431823730469,
 -90.63622283935547,
 -83.77232360839844,
 -76.8875732421875,
 -76.29827117919922,
 -90.11893463134766,
 -69.8428726196289,
 -90.49956512451172,
 -97.02901458740234,
 -78.3626937866211,
 -83.8988037109375,
 -82.5892105102539,
 -79

In [10]:
len(Responses_Test)

1647

In [11]:
# create a csv file with the predictions

import pandas as pd
def Save_Predictions_To_CSV(Ids, Predictions, Output_CSV_Path):
    df = pd.DataFrame({
        'id': Ids,
        'energy': Predictions
    })
        # sort the dataframe by id
    df.sort_values(by='id', inplace=True)
    df.to_csv(Output_CSV_Path, index=False)


    print(f"Predictions saved to {Output_CSV_Path}")

Save_Predictions_To_CSV(IDs_Test, Responses_Test, 'predictions_xgboost.csv')

Predictions saved to predictions_xgboost.csv
